### Probelm 31 : Coin sums

<p>In the United Kingdom the currency is made up of pound (£) and pence (p). There are eight coins in general circulation:</p>
<blockquote>1p, 2p, 5p, 10p, 20p, 50p, £1 (100p), and £2 (200p).</blockquote>
<p>It is possible to make £2 in the following way:</p>
<blockquote>1×£1 + 1×50p + 2×20p + 1×5p + 1×2p + 3×1p</blockquote>
<p>How many different ways can £2 be made using any number of coins?</p>

### Solution

- Greedy solution :

    Try to get all possible combinations where the sum equals the target amount. To do so, wa can for example take all coins and their muluplications ($1\times C_i, 2\times C_i ..$ ) that are lower than the target, create all the possible combinations and for every combination test if its sum equals the target or not. For target = 200, the global set contains (1×1p , 2x1p ...,200×1p, 1×2p , 2x2p ...,100×2p ... £2). Let's calculate the cardinal of this set.


In [68]:
coins_values = [1,2,5,10,20,50,100,200]
cardinal = 0
target = 200
for coin in coins_values :
    cardinal += target//coin
print(f"The global set cardinal : {cardinal}")

The global set cardinal : 377


The number of subset of this cardianl equals $2^{377} =$ 307828173409331868845930000782371982852185463050511302093346042220669701339821957901673955116288403443801781174272. It is a huuuuge number, isn't it ?

This is why Dynamic programing exist !! 

### Dynamic programing
The question is not to give all the possible combinations, but simply calculate **_the number of these combinations_**.

#### 1. Use a single unit. 
We convert all the coins into thier equivalents in pence (£2 = 200p ...). The list of all coins is : `coins_values = [1,2,5,10,20,50,100,200]` and our target is `target = 200`.

#### 2. find a relation between different amounts and coins.
To solve this problem, we will use a great maths operation : **THE SUM**.

We can express every amount as a sum of two amounts (these amount are less than $n$) :
- 6 = 5+1 , 6 = 4+2, 6 = 3+3
- 8 = 7+1 , 8 = 6+2, 8 = 5+3 , 8 = 4+4
- ...

YEES !!  you have it in mind. We can define a recursive function $f(n)$ as $\sum_{i=1}^{n//2}{f(i)*f(n-i)}$ (By defining some extra rules : adding 1 if n is a coin, and f(1)=1, f(2)=2 as stoping rule.)

But waaait wait! if we take 6 = 4+2, a possible combination is 6 = (1+1+1+1) + (1+1) = (1+1+1+1+1+1). For 6 = 3+3, one possible combination is 6 = (1+1+1) + (1+1+1) = (1+1+1+1+1+1). We can see that using this formula will give a false value as it can count the occurence of the same combination in different possible combinations. But the good news is that by using this function we can count all the possible combinations without loosing any possible combination. 

Now let's get rid of this redundancy :

- The  first note is that the result (in a set point of view) won't change if we restrict the sub_amout to the possible coins:
    - 8 = 1+7, 8 = 2+6, 8 = 5+3 
        - 8 = 4+4 is no longer in the set as 4 is not a coin so 4 is necessarly composed of 2 and/or 1 which are given in the other combinations


#### 3.Dynamic programing :

let $coins\_function(c,n)$ be the cardinal of the the set of the combinations where the sum of elements equals n and the maximum value equals c :
$
\begin{equation}
coins\_function(c,n) = Card(S) \ \  s.t: \\  S = \{\omega \in \mathcal{P}(\Omega) : max({\omega}) = c \ and \sum_{x \in \omega} = n\}
\end{equation}
$

```
mermaid
graph LR
A -> B
```

In [ ]:
## DYNAMIC PROGRAMMING
import numpy as np 
import itertools 
coins_values = [1,2,5,10,20,50,100,200]
max_value = 200
coins_matrix = np.zeros((len(coins_values),max_value)).astype(int)
for j in range(max_value) :
    coins_matrix[0,j] = 1
for j in range(1,max_value) :
    target = j+1
    for i_x,x in enumerate(coins_values[1:]):
        i = i_x+1
        value = 0
        if x<target :
            value += sum([coins_matrix[ii,j-x] for ii in range(i+1)])
        elif x == target :
            value = 1
        else :
            break
        coins_matrix[i,j] = value

print(coins_matrix)        
print(coins_matrix[:,max_value-1].sum())
        


## PROBLEM 514
<p>A <b>geoboard</b> (of order <var>N</var>) is a square board with equally-spaced pins protruding from the surface, representing an integer point lattice for coordinates 0 ≤ <var>x</var>,<var>y</var> ≤ <var>N</var>.</p>

<p>John begins with a pinless geoboard. Each position on the board is a hole that can be filled with a pin. John decides to generate a random integer between 1 and <var>N</var>+1 (inclusive) for each hole in the geoboard. If the random integer is equal to 1 for a given hole, then a pin is placed in that hole.</p>

<p>After John is finished generating numbers for all (<var>N</var>+1)<sup>2</sup> holes and placing any/all corresponding pins, he wraps a tight rubberband around the entire group of pins protruding from the board. Let <var>S</var> represent the shape that is formed. <var>S</var> can also be defined as the smallest convex shape that contains all the pins.</p>

<div align="center"><img src="images/p514_geoboard.png" alt="p514_geoboard.png" /></div>

<p>The above image depicts a sample layout for <var>N</var> = 4. The green markers indicate positions where pins have been placed, and the blue lines collectively represent the rubberband. For this particular arrangement, <var>S</var> has an area of 6. If there are fewer than three pins on the board (or if all pins are collinear), <var>S</var> can be assumed to have zero area.</p>

<p>Let E(<var>N</var>) be the expected area of <var>S</var> given a geoboard of order <var>N</var>. For example, E(1) = 0.18750, E(2) = 0.94335, and E(10) = 55.03013 when rounded to five decimal places each.</p>

<p>Calculate E(100) rounded to five decimal places.</p>

### SOLUTION :

It is clear that the expected number of pins is $\sum(1\times (1/(N+1)))$

In [22]:
def f(n):
    if n == 1 :
        return 0.18750
    else :
        return (1+(2*n-1)/((n)**2))*f(n-1)

In [23]:
f(2)

0.328125